# Basic Quantum Gates in Qiskit, Cirq and Pennylane  
**Goals**

1. Prepare single‑qubit states with \(X\) and \(H\).  
2. Create a Bell state using CNOT.  
3. Observe measurement statistics in three SDKs.  
4. Notice the common abstractions (circuits, simulators, counts).

## Observations  

* All three SDKs use the same gate names (`H`, `CNOT`) and produce $|00\rangle$ $|11\rangle$ with roughly equal probability for a Bell state.  
* Qiskit and Cirq supply dedicated histogram helpers; in Pennylane we convert samples manually.  
* The results confirm that unitary gates preserve norm and that measurement collapses the state into the computational basis.

In [ ]:
# Common imports
import numpy as np
import matplotlib.pyplot as plt

## 1  IBM Qiskit

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram, plot_state_city

# 1-qubit: |0> --H--> |+>
qc1 = QuantumCircuit(1, 1)
qc1.h(0)
qc1.measure(0, 0)
backend = Aer.get_backend("qasm_simulator")
counts1 = execute(qc1, backend, shots=1024).result().get_counts()
plot_histogram(counts1)

In [ ]:
# 2-qubit Bell state: H + CNOT
bell = QuantumCircuit(2, 2)
bell.h(0)
bell.cx(0, 1)
bell.measure([0, 1], [0, 1])
counts_bell = execute(bell, backend, shots=1024).result().get_counts()
plot_histogram(counts_bell)

## 2  Google Cirq

In [ ]:
import cirq

# Single qubit |0> --H--> |+>
q = cirq.LineQubit(0)
circuit_cirq = cirq.Circuit(cirq.H(q), cirq.measure(q, key="m"))
sim = cirq.Simulator()
result1 = sim.run(circuit_cirq, repetitions=1024)
cirq.plot_state_histogram(result1)

In [ ]:
# Bell state
q0, q1 = cirq.LineQubit.range(2)
bell_circ = cirq.Circuit(cirq.H(q0), cirq.CNOT(q0, q1),
                         cirq.measure(q0, q1, key="m"))
result_bell = sim.run(bell_circ, repetitions=1024)
cirq.plot_state_histogram(result_bell)

## 3  Pennylane

In [ ]:
import pennylane as qml

dev = qml.device("default.qubit", wires=2, shots=1024)

@qml.qnode(dev)
def bell_pennylane():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

samples = bell_pennylane()
# Convert ±1 samples to bit strings
bits = ((1 - samples[0])//2) * 2 + ((1 - samples[1])//2)
counts = {format(i, "02b"): int(np.sum(bits == i)) for i in range(4)}
plt.bar(counts.keys(), counts.values())
plt.title("Bell state counts (Pennylane)")
plt.show()